<div style="border: 5px solid purple; padding: 10px; margin: 5px">
<b> Svetlana's comment  </b>
    

Hi Andrew, it's Svetlana ([my TripleTen Hub Profile](https://hub.tripleten.com/u/6dee602c)).


Thank you for submitting such a great plan! It has everything you need, so you can move on to the code part! :) 



I recommend using some more robust models apart from Log. Regression. Log Regression is a good start, but it may not be able to provide the desired metric value. 

<hr>

Answers to your questions: 




- Are there any columns that I should remove? ('paperless_billing', for example seems irrelevant to the target we're looking for.)


This is a question you should think about. If the column is indeed irrelevant (id or name columns, for instance, as they do not influence anything), then you definitely do not need it. Regarding `paperless_billing` it's hard to tell for sure. What if it turns out it is useful? You never know what influences the bahavior. Consider using all features first, except for those that cannot influence anything. Then train models without some features and compare the results.


- Is the "Interconnect" company pushing for a specific service to be promoted the most out of all that they provide?

Even if they do, it does not actually change anything in this task.


- Should I train multiple models for this project?


You need to achieve a specific metric value, like in real-life scenarios. If you can do this with one simple model, then it's great! There's no task to train a lot of model. In real life, you get the data and you need to train a model that makes predictions or classifies data correctly in some % of cases. So you main goal is usually to to tran a model (you choose models and the way you train them) that does this. The goal is not to train a specific model or to train several models. The goal is to train a model that yields a desired metric. Decision makers do not care about the models we choose or how many of them we train :)


- Are multiple models typically tried in "real-world" scenarios when working through ML tasks?


Of course. Real tasks are quite complicated and the data is huge and dirty. You sometimes deal with tasks nobody has done before, so you just don't know which model will guarantee a good result. So you have to try different models.


- How much parameter tuning should I try before leaving the project?

The task is about metric score, not about tuning. If you achieve the desired metric without tuning, then it's fine. 
</div>

# Retaining Interconnect Clients

The company "Interconnect" would like to be able to accurately forecast their churn of clients and hopefully reduce their churn rate. Their plan of action is to offer specialized promotions and plan options to those customers who are identified as possibly terminating their contracts. 
<br>

The goal for this project is to train a model to accurately identify customers who might be deciding to leave the company so that Interconnect's marketing team can attempt to convince them to hold onto their contract and reduce churn rate.

## Initialization

In [1]:
import pandas as pd


In [2]:
# Constants


## Preprocessing & EDA

### Columns Formatting

In [3]:
# Importing Data

con_data = pd.read_csv('/datasets/final_provider/contract.csv')
per_data = pd.read_csv('/datasets/final_provider/personal.csv')
int_data = pd.read_csv('/datasets/final_provider/internet.csv')
pho_data = pd.read_csv('/datasets/final_provider/phone.csv')

In [4]:
# Formatting Columns

con_data.columns = con_data.columns.str.replace(r'(?<=[a-z0-9])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])', '_', regex=True).str.lower()
per_data.columns = per_data.columns.str.replace(r'(?<=[a-z0-9])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])', '_', regex=True).str.lower()
int_data.columns = int_data.columns.str.replace(r'(?<=[a-z0-9])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])', '_', regex=True).str.lower()
pho_data.columns = pho_data.columns.str.replace(r'(?<=[a-z0-9])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])', '_', regex=True).str.lower()

In [5]:
# Validating changes to column names

print("Contract Data")
print(con_data.columns)
print()

print("Personal Data")
print(per_data.columns)
print()

print("Internet Data")
print(int_data.columns)
print()

print("Phone Data")
print(pho_data.columns)

Contract Data
Index(['customer_id', 'begin_date', 'end_date', 'type', 'paperless_billing',
       'payment_method', 'monthly_charges', 'total_charges'],
      dtype='object')

Personal Data
Index(['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents'], dtype='object')

Internet Data
Index(['customer_id', 'internet_service', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv',
       'streaming_movies'],
      dtype='object')

Phone Data
Index(['customer_id', 'multiple_lines'], dtype='object')


<div style="border: 2px solid black; padding: 10px; margin: 10px">

I started with formatting the columns in each data set to avoid any unnecessary complications that could arise later due to formatting issues. Adjusted the Regex Code above to make sure the "ID" and "TV" were separated as one word and not formatted with any extra underscores.
<br>

Columns look great! Moving on to finding duplicates.

</div>

### Duplicates

#### Contract Data

In [14]:
# Contract Data

print("Contract Data")
print("=========================================================================")

duplicate_summary_con = pd.DataFrame({
    "duplicates_count": [con_data[col].duplicated().sum() for col in con_data.columns],
    "duplicates_percent": [con_data[col].duplicated().mean() * 100 for col in con_data.columns],
    "unique_values": [con_data[col].nunique() for col in con_data.columns],
    "total_rows": len(con_data)
}, index=con_data.columns)

duplicate_summary_con

Contract Data


,duplicates_count,duplicates_percent,unique_values,total_rows
customer_id,0,0.000000,7043,7043
begin_date,6966,98.906716,77,7043
end_date,7038,99.929008,5,7043
type,7040,99.957405,3,7043
paperless_billing,7041,99.971603,2,7043
payment_method,7039,99.943206,4,7043
monthly_charges,5458,77.495385,1585,7043
total_charges,512,7.269629,6531,7043


<div style="border: 2px solid black; padding: 10px; margin: 10px">

While looking at our duplicate data for the **Contract Data** set, we see there are a high rate of duplicates for `begin_date`, `end_date`, `type`, `paperless_billing`, and `payment_method`. Each of these, though, also only have a few unique values (except for `begin_date`). The others with 2-5 unique values have duplicates because of the nature of the amount of possibilities that exist for that column. In terms of `begin_date`, there could have been a promotion going on that was able to capture more customers at one time, so seeing a high percentage in this column is also expected.
<br>

We're also confirming that there are 7,043 unique customer ID's.
<br>

For this dataset, we will leave all the duplicates present as they are important pieces of information that we'll use to train our model.

</div>

#### Personal Data

In [15]:
# Personal Data

print("Personal Data")
print("=========================================================================")

duplicate_summary_per = pd.DataFrame({
    "duplicates_count": [per_data[col].duplicated().sum() for col in per_data.columns],
    "duplicates_percent": [per_data[col].duplicated().mean() * 100 for col in per_data.columns],
    "unique_values": [per_data[col].nunique() for col in per_data.columns],
    "total_rows": len(per_data)
}, index=per_data.columns)

duplicate_summary_per

Personal Data


,duplicates_count,duplicates_percent,unique_values,total_rows
customer_id,0,0.000000,7043,7043
gender,7041,99.971603,2,7043
senior_citizen,7041,99.971603,2,7043
partner,7041,99.971603,2,7043
dependents,7041,99.971603,2,7043


<div style="border: 2px solid black; padding: 10px; margin: 10px">

In our **Personal Data** set, we're again seeing unique `customer_id`'s, and based on the number of unique values, and the information they represent, we can leave these duplicates for this dataset as well since they hold important information that can help inform our model later on.

</div>

#### Internet Data

In [16]:
# Internet Data

print("Internet Data")
print("=========================================================================")

duplicate_summary_int = pd.DataFrame({
    "duplicates_count": [int_data[col].duplicated().sum() for col in int_data.columns],
    "duplicates_percent": [int_data[col].duplicated().mean() * 100 for col in int_data.columns],
    "unique_values": [int_data[col].nunique() for col in int_data.columns],
    "total_rows": len(int_data)
}, index=int_data.columns)

duplicate_summary_int

Internet Data


,duplicates_count,duplicates_percent,unique_values,total_rows
customer_id,0,0.000000,5517,5517
internet_service,5515,99.963748,2,5517
online_security,5515,99.963748,2,5517
online_backup,5515,99.963748,2,5517
device_protection,5515,99.963748,2,5517
tech_support,5515,99.963748,2,5517
streaming_tv,5515,99.963748,2,5517
streaming_movies,5515,99.963748,2,5517


<div style="border: 2px solid black; padding: 10px; margin: 10px">

We're seeing a similar scenario here in our **Internet Data** set; the duplicates are intentional and we will keep them for training.

</div>

#### Phone Data

In [17]:
# Phone Data

print("Phone Data")
print("=========================================================================")

duplicate_summary_pho = pd.DataFrame({
    "duplicates_count": [pho_data[col].duplicated().sum() for col in pho_data.columns],
    "duplicates_percent": [pho_data[col].duplicated().mean() * 100 for col in pho_data.columns],
    "unique_values": [pho_data[col].nunique() for col in pho_data.columns],
    "total_rows": len(pho_data)
}, index=pho_data.columns)

duplicate_summary_pho

Phone Data


,duplicates_count,duplicates_percent,unique_values,total_rows
customer_id,0,0.000000,6361,6361
multiple_lines,6359,99.968558,2,6361


<div style="border: 2px solid black; padding: 10px; margin: 10px">

The duplicates in our **Phone Data** set are also intentional and will be left for training.
<br>

Next, we will uncover any missing values in our data.

</div>

### Missing Values

#### Contract Data

In [21]:
# Contract Data

print("Contract Data")
print("=========================================================================")

na_sum_con = pd.DataFrame({
    "missing_count": con_data.isna().sum(),
    "missing_percent": (con_data.isna().mean() * 100).round(2),
    "unique_values": con_data.nunique(),
    "total_rows": len(con_data)
})

# Sorting values by missing value count
na_sum_con = na_sum_con.sort_values(by="missing_count", ascending=False)

na_sum_con

Contract Data


,missing_count,missing_percent,unique_values,total_rows
customer_id,0,0.0,7043,7043
begin_date,0,0.0,77,7043
end_date,0,0.0,5,7043
type,0,0.0,3,7043
paperless_billing,0,0.0,2,7043
payment_method,0,0.0,4,7043
monthly_charges,0,0.0,1585,7043
total_charges,0,0.0,6531,7043


<div style="border: 2px solid black; padding: 10px; margin: 10px">

No missing values.

</div>

#### Personal Data

In [19]:
# Personal Data

print("Personal Data")
print("=========================================================================")

na_sum_per = pd.DataFrame({
    "missing_count": per_data.isna().sum(),
    "missing_percent": (per_data.isna().mean() * 100).round(2),
    "unique_values": per_data.nunique(),
    "total_rows": len(per_data)
})

# Sorting values by missing value count
na_sum_per = na_sum_per.sort_values(by="missing_count", ascending=False)

na_sum_per

Personal Data


,missing_count,missing_percent,unique_values,total_rows
customer_id,0,0.0,7043,7043
gender,0,0.0,2,7043
senior_citizen,0,0.0,2,7043
partner,0,0.0,2,7043
dependents,0,0.0,2,7043


<div style="border: 2px solid black; padding: 10px; margin: 10px">

No missing values.

</div>

#### Internet Data

In [20]:
# Internet Data

print("Internet Data")
print("=========================================================================")

na_sum_int = pd.DataFrame({
    "missing_count": int_data.isna().sum(),
    "missing_percent": (int_data.isna().mean() * 100).round(2),
    "unique_values": int_data.nunique(),
    "total_rows": len(int_data)
})

# Sorting values by missing value count
na_sum_int = na_sum_int.sort_values(by="missing_count", ascending=False)

na_sum_int

Internet Data


,missing_count,missing_percent,unique_values,total_rows
customer_id,0,0.0,5517,5517
internet_service,0,0.0,2,5517
online_security,0,0.0,2,5517
online_backup,0,0.0,2,5517
device_protection,0,0.0,2,5517
tech_support,0,0.0,2,5517
streaming_tv,0,0.0,2,5517
streaming_movies,0,0.0,2,5517


<div style="border: 2px solid black; padding: 10px; margin: 10px">

No missing values.

</div>

#### Phone Data

In [22]:
# Phone Data

print("Phone Data")
print("=========================================================================")

na_sum_pho = pd.DataFrame({
    "missing_count": pho_data.isna().sum(),
    "missing_percent": (pho_data.isna().mean() * 100).round(2),
    "unique_values": pho_data.nunique(),
    "total_rows": len(pho_data)
})

# Sorting values by missing value count
na_sum_pho = na_sum_pho.sort_values(by="missing_count", ascending=False)

na_sum_pho

Phone Data


,missing_count,missing_percent,unique_values,total_rows
customer_id,0,0.0,6361,6361
multiple_lines,0,0.0,2,6361


<div style="border: 2px solid black; padding: 10px; margin: 10px">

We have no missing values in all of our datasets.
<br>

We'll now move on to combining all 4 dataframes into one and then continue to data type validation, date parsing, etc.

</div>

### Combining Dataframes

In [37]:
# Before combining, ensuring that 'customer_id' is the same datatype across all dataframes

print("Contract Data")
print(con_data['customer_id'].dtype)
print()

print("Personal Data")
print(per_data['customer_id'].dtype)
print()

print("Internet Data")
print(int_data['customer_id'].dtype)
print()

print("Phone Data")
print(pho_data['customer_id'].dtype)
print()

Contract Data
object

Personal Data
object

Internet Data
object

Phone Data
object



In [31]:
# Now, combining all four dataframes on 'customer_id' on an 'outer' join
# Intention is to include as much data as possible and we'll decide what to do with
# the missing values afterwards after identifying them.

df_merged = (
    con_data
    .merge(per_data, on="customer_id", how="outer")
    .merge(int_data, on="customer_id", how="outer")
    .merge(pho_data, on="customer_id", how="outer")
)

In [38]:
# Checking on duplicates for merged DF

df_merged.duplicated().sum()

0

In [40]:
# Reviewing missing values after merge

print("Merged DF")
print("=========================================================================")

na_sum_merged = pd.DataFrame({
    "missing_count": df_merged.isna().sum(),
    "missing_percent": (df_merged.isna().mean() * 100).round(2),
    "unique_values": df_merged.nunique(),
    "total_rows": len(df_merged)
})

# Sorting values by missing value count
na_sum_merged = na_sum_merged.sort_values(by="missing_count", ascending=False)

na_sum_merged

Merged DF


,missing_count,missing_percent,unique_values,total_rows
streaming_movies,1526,21.67,2,7043
streaming_tv,1526,21.67,2,7043
tech_support,1526,21.67,2,7043
device_protection,1526,21.67,2,7043
online_backup,1526,21.67,2,7043
online_security,1526,21.67,2,7043
internet_service,1526,21.67,2,7043
multiple_lines,682,9.68,2,7043
begin_date,0,0.00,77,7043
dependents,0,0.00,2,7043


<div style="border: 2px solid black; padding: 10px; margin: 10px">

All the missing values in our merged DF have only two possibilities. Since the ID's for these customers were not present in those dataframes we'll fill all the missing values with the equivalent for "No", that the customer does not have the feature of that column.

</div>

In [43]:
# Filling all missing values in the df with "No"

df_merged = df_merged.fillna("No")

# Validating no more missing values

df_merged.isna().sum()

customer_id          0
begin_date           0
end_date             0
type                 0
paperless_billing    0
payment_method       0
monthly_charges      0
total_charges        0
gender               0
senior_citizen       0
partner              0
dependents           0
internet_service     0
online_security      0
online_backup        0
device_protection    0
tech_support         0
streaming_tv         0
streaming_movies     0
multiple_lines       0
dtype: int64

### Data Type Validation

In [44]:
# Merged Data

print("Merged Data")
print("=========================================================================")

df_merged.info()
print()

df_merged

Merged Data
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        7043 non-null   object 
 1   begin_date         7043 non-null   object 
 2   end_date           7043 non-null   object 
 3   type               7043 non-null   object 
 4   paperless_billing  7043 non-null   object 
 5   payment_method     7043 non-null   object 
 6   monthly_charges    7043 non-null   float64
 7   total_charges      7043 non-null   object 
 8   gender             7043 non-null   object 
 9   senior_citizen     7043 non-null   int64  
 10  partner            7043 non-null   object 
 11  dependents         7043 non-null   object 
 12  internet_service   7043 non-null   object 
 13  online_security    7043 non-null   object 
 14  online_backup      7043 non-null   object 
 15  device_protection  7043 non-null   object 
 16  tech_support

,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,gender,senior_citizen,partner,dependents,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,multiple_lines
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,Female,0,Yes,No,DSL,No,Yes,No,No,No,No,No
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5,Male,0,No,No,DSL,Yes,No,Yes,No,No,No,No
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,Male,0,No,No,DSL,Yes,Yes,No,No,No,No,No
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,Male,0,No,No,DSL,Yes,No,Yes,Yes,No,No,No
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,Female,0,No,No,Fiber optic,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,2018-02-01,No,One year,Yes,Mailed check,84.80,1990.5,Male,0,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,Yes
7039,2234-XADUH,2014-02-01,No,One year,Yes,Credit card (automatic),103.20,7362.9,Female,0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Yes
7040,4801-JZAZL,2019-03-01,No,Month-to-month,Yes,Electronic check,29.60,346.45,Female,0,Yes,Yes,DSL,Yes,No,No,No,No,No,No
7041,8361-LTMKD,2019-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Mailed check,74.40,306.6,Male,1,Yes,No,Fiber optic,No,No,No,No,No,No,Yes


<div style="border: 2px solid black; padding: 10px; margin: 10px">

Contract Data `info()` and data displayed to validate content and datatype match. After review, we're seeing multiple changes to be made to our data before we can train our model:
<br><br>

**Datetime Data**
<ol>
    <li>First, we need to convert the 'begin_date' and 'end_date' columns to datetime</li>
    <li>We'll then create a new column where we will calculate total contract duration</li>
    <li>Next, we'll parse out the month, day and year from each 'begin_date' to help with informing seasonality</li>
    <li>Lastly, we'll create a column with days since the contract was ended</li>
</ol>

**Churn Data**
<ul>
    <li>We'll create this column with numerical variables for better training</li>
</ul>

**Total Charges**
<ul>
    <li>Data type for this column is in "object" type, we will convert this to 'float64'</li>
</ul>

**Encoding**
<ul>
    <li>We'll then have to encode all the numerical and categorical features before training</li>
</ul>

</div>

<div style="border: 2px solid red; padding: 10px; margin: 10px">
<b>My Project Plan:</b>

<ol>
    <li>I'm going to complete the list of changes above to the merged dataframe.</li>
    <li>After, I'm going to display some graphs and visuals to help illustrate the data</li>
    <ul>
        <li>Distribution of male/female, senior citizens? (It might be interesting to know the majority gender that subscribes to the company for marketing, etc.)</li>
        <li>Distribution of times of the year that people start contracts and end them. (This could help inform the effectiveness of past promotions or to plan future promotions to help boost contract rates.)</li>
        <li>Distribution of monthly charges and if the customer has cancelled their contract or not. (This might give some insight as to the amount of the contract that turns clients away from the company services and where the threshold should be adjusted to, if at all.)</li>
    </ul>
    <li>Then I will split the features and target to start training the model. (I'll be using <b>Logistic Regression</b> as the model for this project.</li>
    <li>I'll calculate the AUC-ROC, and Accuracy after testing the model on the validation set.</li>
</ol>
<br><br>

<b>List of Clarifying Questions:</b>
<ol>
    <li>Are there any columns that I should remove? ('paperless_billing', for example seems irrelevant to the target we're looking for.)</li>
    <li>Is the "Interconnect" company pushing for a specific service to be promoted the most out of all that they provide?</li>
    <li>Should I train multiple models for this project?</li>
    <li>Are multiple models typically tried in "real-world" scenarios when working through ML tasks?</li>
    <li>How much parameter tuning should I try before leaving the project?</li>
</ol>
<br><br>

Thank you!
</div>

## EDA

## Model Training

## Conclusions

<div style="border: 2px solid black; padding: 10px; margin: 10px">

Conclusion goes here.

</div>